In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from prediction_utils.pytorch_utils.metrics import StandardEvaluator, xauc
from sklearn.metrics import roc_auc_score


In [12]:
sigma = 0.01

na0 = 1000
na1 = 100
nb0 = 1000
nb1 = 100

pa0 = 0.1
pa1 = 0.2
pb0 = 0.3
pb1 = 0.4

pred_probs_a0 = pa0*np.ones(na0) + (np.random.randn(na0) * sigma)
pred_probs_a1 = pa1*np.ones(na1) + (np.random.randn(na1) * sigma)
pred_probs_b0 = pb0*np.ones(nb0) + (np.random.randn(nb0) * sigma)
pred_probs_b1 = pb1*np.ones(nb1) + (np.random.randn(nb1) * sigma)

pya = na1/(na1+na0)
pyb = nb1/(nb1+nb0)

pay1 = na1 / (na1 + nb1)
pay0 = na0 / (na0 + nb0)

expected_auc = 1 - (pay1 * (1 - pay0))
print(expected_auc)

0.75


In [13]:
df = pd.concat(
    [
        pd.DataFrame(
            {
                'pred_probs': pred_probs_a0,
                'labels': 0,
                'group': 'a',
            }
        ),
        pd.DataFrame(
            {
                'pred_probs': pred_probs_a1,
                'labels': 1,
                'group': 'a',
            }
        ),
        pd.DataFrame(
            {
                'pred_probs': pred_probs_b0,
                'labels': 0,
                'group': 'b',
            }
        ),
        pd.DataFrame(
            {
                'pred_probs': pred_probs_b1,
                'labels': 1,
                'group': 'b',
            }
        )
    ]
)

In [14]:
print(roc_auc_score(df.labels, df.pred_probs))
pooled_xauc_a = pya * xauc(df.labels, df.pred_probs, df.group, 'a', the_label=1) + (1-pya)*xauc(df.labels, df.pred_probs, df.group, 'a', the_label=0)
pooled_xauc_b = pyb * xauc(df.labels, df.pred_probs, df.group, 'b', the_label=1) + (1-pyb)*xauc(df.labels, df.pred_probs, df.group, 'b', the_label=0)
print(pooled_xauc_a)
print(pooled_xauc_b)

0.75
0.9545454545454545
0.5454545454545454


In [15]:
evaluator = StandardEvaluator(metrics=['auc'])
evaluator.get_result_df(df, group_var_name='group')

,metric,group,performance
0,auc,a,1.00
1,auc,b,1.00
0,auc,overall,0.75
